# Go Big or Go Home Part 2 - Working and Visualising on cluster <img align="right" src="../../resources/csiro_easi_logo.png">

In this notebook we finally do our larger area. We're going to need some better visualisation tools and it would be great not to bring the results back to the Jupyter notebook but to leverage the dask clusters resources during visualisation. We'll be using some dask-aware visualiation libraries (holoviews and datashader) to do the heavy lifting.

Let's begin by starting up our cluster and sizing it appropriately to our computational task.

## Time to go big!

All the code here is the same as the conclusion from the previous notebook, except we'll make the cluster bigger with 10 workers instead of 4. We'll also make the masking and NDVI calculation into a python function since we won't be making any changes to that now.

We'll use the same ROI and time period for this run and we're using all the techniques so far to reduce the computation time:
1. Dask chunk size selection
2. Only loading the measurements we intend on using in this calculation to save on the task graph optimisation time

In [ ]:
# Initialize the Gateway client
from dask.distributed import Client
from dask_gateway import Gateway

number_of_workers = 10 

gateway = Gateway()

clusters = gateway.list_clusters()
if not clusters:
    print('Creating new cluster. Please wait for this to finish.')
    cluster = gateway.new_cluster()
else:
    print(f'An existing cluster was found. Connecting to: {clusters[0].name}')
    cluster=gateway.connect(clusters[0].name)

cluster.scale(number_of_workers)

client = cluster.get_client()
client

In [ ]:
import pyproj
pyproj.set_use_global_context(True)

import git
import sys, os
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from dask.distributed import Client, LocalCluster
import datacube
from datacube.utils import masking
from datacube.utils.aws import configure_s3_access

# EASI defaults
os.environ['USE_PYGEOS'] = '0'
repo = git.Repo('.', search_parent_directories=True).working_tree_dir
if repo not in sys.path: sys.path.append(repo)
from easi_tools import EasiDefaults, notebook_utils
easi = EasiDefaults()

In [ ]:
dc = datacube.Datacube()
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

In [ ]:
# Get the centroid of the coordinates of the default extents
central_lat = sum(easi.latitude)/2
central_lon = sum(easi.longitude)/2
# central_lat = -42.019
# central_lon = 146.615

# Set the buffer to load around the central coordinates
# This is a radial distance for the bbox to actual area so bbox 2x buffer in both dimensions
buffer = 0.8

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

# Data product
products = easi.product('landsat')

# Set the date range to load data over
set_time = easi.time
set_time = (set_time[0], parse(set_time[0]) + relativedelta(years=1))
# set_time = ("2021-01-01", "2021-12-31")

# Selected measurement names (used in this notebook)
alias = easi.aliases('landsat')
measurements = [alias[x] for x in ['qa_band', 'red', 'nir']]

# Set the QA band name and mask values
qa_band = alias['qa_band']
qa_mask = easi.qa_mask('landsat')

# Set the resampling method for the bands
resampling = {qa_band: "nearest", "*": "average"}

# Set the coordinate reference system and output resolution
set_crs = easi.crs('landsat')  # If defined, else None
set_resolution = easi.resolution('landsat')  # If defined, else None
# set_crs = "epsg:3577"
# set_resolution = (-30, 30)

# Set the scene group_by method
group_by = "solar_day"

In [ ]:
def masked_seasonal_ndvi(dataset):
    # Identify pixels that are either "valid", "water" or "snow"
    cloud_free_mask = masking.make_mask(dataset[qa_band], **qa_mask)
    # Apply the mask
    cloud_free = dataset.where(cloud_free_mask)

    # Calculate the components that make up the NDVI calculation
    band_diff = cloud_free[alias['nir']] - cloud_free[alias['red']]
    band_sum = cloud_free[alias['nir']] + cloud_free[alias['red']]
    # Calculate NDVI
    ndvi = None
    ndvi = band_diff / band_sum

    return ndvi.groupby("time.season").mean("time")  # Calculate the seasonal mean

dataset = None # clear results from any previous runs
dataset = dc.load(
            product=products,
            x=study_area_lon,
            y=study_area_lat,
            time=set_time,
            measurements=measurements,
            resampling=resampling,
            output_crs=set_crs,
            resolution=set_resolution,
            dask_chunks = {"time":2, "x":3072, "y":3072},
            group_by=group_by,
        )

ndvi_unweighted = masked_seasonal_ndvi(dataset)

In [ ]:
print(f"dataset size (GiB) {dataset.nbytes / 2**30:.2f}")
print(f"ndvi_unweighted size (GiB) {ndvi_unweighted.nbytes / 2**30:.2f}")

In [ ]:
# client.wait_for_workers(n_workers=10)  # Before release 2023.10.0
client.sync(client._wait_for_workers,n_workers=10) # Since release 2023.10.0

In [ ]:
cluster

In [ ]:
%%time
actual_result = ndvi_unweighted.compute()

You'll notice the computation time is slightly faster with more workers - we're IO bound so more workers means more available IO bandwidth and threads. It's not 2-3 x faster though - we're wasting a lot of resources because we can't actually use all of that extra power.

> __Tip__: More isn't always better. Be mindful of your computational resource usage and cost. This size cluster is a tremendous waste for this size computational job. Size things appropriately.

And visualise the result

In [ ]:
actual_result.sel(season='DJF').plot()

We'll save the coordinates of this section from the array (as slices) so we can use them later for visualising the same ROI from a larger dataset.

In [ ]:
x_slice = slice(ndvi_unweighted.x[0], ndvi_unweighted.x[-1])
y_slice = slice(ndvi_unweighted.y[0], ndvi_unweighted.y[-1])

## Now for a bigger area

Let's change the area extent to about 4 degrees square.


In [ ]:
# Compute the bounding box for the study area
buffer = 2
# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

In [ ]:
# Check the map below to see if you are including the area that you want. For this example, it would be best to not include too much water.
from dea_tools.plotting import display_map
display_map(study_area_lon, study_area_lat)

In [ ]:
dataset = None # clear results from any previous runs
dataset = dc.load(
            product=products,
            x=study_area_lon,
            y=study_area_lat,
            time=set_time,
            measurements=measurements,
            resampling=resampling,
            output_crs=set_crs,
            resolution=set_resolution,
            dask_chunks = {"time":2, "x":3072, "y":3072},
            group_by=group_by,
        )

ndvi_unweighted = masked_seasonal_ndvi(dataset)

Before we compute anything let's take a look at our result's shape and size

In [ ]:
print(f"dataset size (GiB) {dataset.nbytes / 2**30:.2f}")
print(f"ndvi_unweighted size (GiB) {ndvi_unweighted.nbytes / 2**30:.2f}")

This is now much bigger!

The result is getting on the large size for the notebook node __so we will need to pay attention to _data locality_ and the size of results being processed__. The cluster has a LOT more memory than the _notebook node_; bring too much back to the notebook and the notebook will crash.

> __Tip__: Be mindful of the size of the results and their _data locality_. 

Now let's check the _shape_, _tasks_ and _chunks_

In [ ]:
dataset

Looking at the `red` data variable we can see about 50 GiB for the array, 36 MiB per chunk and 5526 tasks. Noting the `nir` and `qa_band` will be similarly shaped and size.

The number of tasks is climbing so we can expect an increase in _task graph optimisation_ time.

Chunk size and tasks seems okay, but we will monitor the _dask dashboard_ in case there are issues with temporaries causing _workers_ to _spill to disk_ if memory is too full.

_The chunking is resulting in some slivers, particularly on the y axis._ Let's modify the y chunk size so these slivers don't exist as its blowing out the tasks and is likely unnecessary. We will calculate the x and y chunk sizes below to get a nice fit. _Make sure to check the chunk size afterwards to make sure it doesn't get too large. If it does we can make the chunks smaller to reduce slivers too._

In [ ]:
from math import ceil

y_chunks = ceil(dataset.dims['y']/5)
y_chunks

In [ ]:
dataset = None # clear results from any previous runs
dataset = dc.load(
            product=products,
            x=study_area_lon,
            y=study_area_lat,
            time=set_time,
            measurements=measurements,
            resampling=resampling,
            output_crs=set_crs,
            resolution=set_resolution,
            dask_chunks = {"time":2, "x":3072, "y":y_chunks},
            group_by=group_by,
        )

ndvi_unweighted = masked_seasonal_ndvi(dataset)

Now recheck our chunk size and tasks for `red`

In [ ]:
dataset

Very marginal increase in _memory per chunk_ but the _tasks_ have dropped from 5526 to 4661. Note that this occurs for every measurement and operation so the benefit is significant.

In [ ]:
ndvi_unweighted

Total task count is sub 100_000 so should be okay but _task graph optimisation_ will take a while. Resulting array is a bit big for the notebook node as stated previously.

The shape spatially is `y:15686, x:13707`. Standard plots aren't going to work very well for visualising the result in the notebook and the result uses a fair amount of memory so we'll need a different approach.

For now let's visualize the same ROI as the small area before. We stashed that ROI in `x_slice, y_slice`.

__If you haven't already, open the dask dashboard so you can watch the cluster make progress__

The code to do this visualisation is basically the same as before except now we specify a slice

In [ ]:
%%time
ndvi_unweighted.sel(season='DJF', x=x_slice, y=y_slice).compute().plot(robust=True, size=6, aspect='equal')

The computation time is relatively short since we are only materialising the result for a subset of the overall dataset.

## Visualising all of the data

To visualise all of the data we will make use of the dask cluster and some dask-aware visualisation capabilties from `holoviews` and `datashader` python libraries. These libraries provide an _interactive_ visualisation capability that leaves the large datasets on the cluster and transmits only the final visualisation to the Jupyter notebook. This is done on the fly so the user can zoom and pan about the dataset in all dimensions and the dask cluster will scale data to fit in the viewport automatically. Details of how this is done and advanced features available is beyond the scope of this dask and ODC course but the manuals are extensive and the basic example here both powerful and useful.

> __Tip__: The [datashader pipeline](https://datashader.org/getting_started/Pipeline.html) page provides an excellent summary of what's going on.

### `compute()` and `persist()`

The first thing we will do is `persist()` the results of our calculation to the cluster. This will materialise the results but will keep the result on the cluster (so all lazy tasks are calculated, just like `compute()` but _data locality_ remains on the cluster). This will ensure the result is readily available for the visualisation. The cluster has plenty of (distributed) memory so there is no reason not to materialise the result on the cluster.

`persist()` is non-blocking so will return just as soon as _task graph optimisation_ (which is performed in the notebook kernel) is complete. Run the next cell and you will see it takes a few seconds to do _task graph optimisation_, and once that is complete the Jupyter notebook will be available for use again. At the same time the _dask dashboard_ will show tasks running as the result is computed and left on the cluster.

In [ ]:
%%time
on_cluster_result = ndvi_unweighted.persist()
# wait(on_cluster_result)
on_cluster_result

The `on_cluster_result` will continue to show as a dask array on the cluster - not actual results. Think of it as a handle that links the _Jupyter client_ to the result on the _dask cluster_.

The cluster will start the computation, but we can continue working in the notebook. Let's import a new visualization library: `hvplot`. We'll be using `datashader.rasterize` via `hvplot` to handle the visualisation of the full dataset which has many more pixels than what what is being displayed in the notebook. `hvplot.xarray` makes visualising `xarray` data a very natural experience, so the code is quite simple, a lot is taken care of for you.

Notice also there are no bounds set on the dataset, we are viewing the entire result, including the _season_ dimension. `hvplot` will provide an interface for pan, zoom and season selection and you can use the mouse to move around the data.

>__Tip:__ Keep watching your Dask dashboard to see how the calculations are progressing.

In [ ]:
# quick calculation so the interactive UI is no more than 700 pixels wide and maintains aspect ratio.
aspect = on_cluster_result.sizes['y']/on_cluster_result.sizes['x']
width = 700
height = int(width*aspect)

The next cell will display the result - when its ready. The `rasterize` function will calculate a representative pixel for display from the full array on the dask cluster. If you monitor the dashboard you will see small bursts of activity across the workers and quite some waiting whilst data transfers occur to bring all the summary information back and transmit it to the Jupyter notebook. It's a large dataset, only the pixels you can see on the screen are sent to your web browser.

You can use the controls on the right to pan and zoom around the full image. If you zoom in, `rasterize` will take a moment to generate a new summary for the current zoom level and show more or less detail. Similarly for panning.

In [ ]:
import hvplot.xarray 
import xarray as xr
on_cluster_result.hvplot.image(groupby='season', rasterize=True).opts(
        title="NDVI Seasonal Mean",
        cmap="RdYlGn", # NDVI more green the larger the value. 
        clim=(-0.3, 0.8), # we'll clamp the range for visualisation to enhance the visualisation
        colorbar=True,
        frame_width=width,
        frame_height=height
    )

# Be a good dask user - Clean up the cluster resources

Disconnecting your client is good practice, but the cluster will still be up so we need to shut it down as well

In [ ]:
client.close()

In [ ]:
cluster.shutdown()